In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv("melb_data.csv")

# Select target
y = data.Price

# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

Create a function that trains random forest models and gives back their MAE

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

## Dealing with Missings

 Does the dataset have a lot of missing values, or just a few? Would we lose much information if we completely ignored the columns with missing entries?

If there are relatively few missing entries in the data (observe e.g column with the greatest percentage of missing values), we can expect that dropping columns is unlikely to yield good results. This is because we'd be throwing away a lot of valuable data, and so imputation will likely perform better.

Given that there are so few missing values in the dataset, we'd expect imputation to perform better than dropping columns entirely. However, sometimes dropping columns performs slightly better! While this can probably partially be attributed to noise in the dataset, another potential explanation is that the imputation method is not a great match to this dataset. That is, maybe instead of filling in the mean value, it makes more sense to set every missing value to a value of 0, to fill in the most frequently encountered value, or to use some other method. For instance, consider a feature which indicates the year that the garage was built. It's likely that in some cases, a missing value could indicate a house that does not have a garage. Does it make more sense to fill in the median value along each column in this case? Or could we get better results by filling in the minimum value along each column? It's not quite clear what's best in this case, but perhaps we can rule out some options immediately - for instance, setting missing values in this column to 0 is likely to yield horrible results!

In [3]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


In approach 1 complete columns are deleted (instead of rows).

Deleting an entire column might not be the best approach unless you see for example the information contained in there is useless:
 for example 90% of the data is missing or using other mechanisms like PCA that would tell you these might not be relevant.

##### Approach 1: Drop the Missings (drop whole feature if some values are missing)

In [4]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


##### Approach 2: Impute missings with mean value of each column

In [5]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
# This learns the needed parameter from X (here: mean) and then transforms the data (here: fill in mean for missing values)
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))
# the validation data is imputed with the mean of the training data

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
178166.46269899711


Es ist auch möglich verschiedene Imputation Techniken zu verbinden:

In [6]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('min', SimpleImputer(strategy='constant', fill_value=X_train['Car'].min()), ['Car']),
        ('median', SimpleImputer(strategy='median'), ['YearBuilt']),
        #('most frequent', SimpleImputer(strategy='most_frequent'), ['BuildingArea'])
        ('drop', 'drop', ['BuildingArea'] )
    ],
    remainder='passthrough'  # alle anderen Spalten bleiben erhalten
)

imputed_X_train2 = pd.DataFrame(preprocessor.fit_transform(X_train))
# This learns the needed parameter from X (here: mean) and then transforms the data (here: fill in mean for missing values)
imputed_X_valid2 = pd.DataFrame(preprocessor.transform(X_valid))
# the validation data is imputed with the mean of the training data

# Imputation removed column names; put them back
imputed_X_train2.columns = X_train.columns.drop('BuildingArea')
imputed_X_valid2.columns = X_valid.columns.drop('BuildingArea')

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train2, imputed_X_valid2, y_train, y_valid))

MAE from Approach 2 (Imputation):
182014.60704467352


##### Approach 3: Impute as in 2 and keep track of what was imputed

In [7]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


##### Approach 4: Deleting rows

In [8]:
row_red_X_train = X_train.dropna(axis=0)
row_red_y_train = y_train.loc[row_red_X_train.index]
print("MAE from Approach 4 (Drop rows with missing values):")
print(score_dataset(row_red_X_train, X_valid, row_red_y_train, y_valid))

MAE from Approach 4 (Drop rows with missing values):
224328.82422680414


jupyter nbconvert --to webpdf missing_values.ipynb